In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import os
import pickle
import numpy as np
import math


In [ ]:
!nvidia-smi

Mon Nov 29 18:54:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:84:00.0 Off |                  N/A |
| 24%   34C    P8    11W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:85:00.0 Off |                  N/A |
| 28%   

In [ ]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
    
    def __init__(self, path):

        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')
            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,1:2])

        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

    def __len__(self):
        return self.data.shape[0]

   
    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = self.label[idx].astype(float)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }

        return batch

In [ ]:
dataset = DeapS2SDatasetClassification('data_preprocessed_python')

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
train_ind = int(0.8 * len(dataset))
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

print(len(train_set))
print(len(val_set))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=12, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=12, shuffle=False, pin_memory=True)

1024
256


In [ ]:
def classification_report(pred,actual,best_class_weights):
    acc = round(best_class_weights[0]*accuracy_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),3)
    precision = round(best_class_weights[1]*precision_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    recall = round(best_class_weights[2]*recall_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    f1score = round(best_class_weights[3]*f1_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
    return acc,precision,recall,f1score

In [ ]:
#defining the models and their architectures
class Encoder(nn.Module):

#this class will initialize the models with the desired architecture
    def __init__(self, input_size, embed_size,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        
# defining lstm and using bidirectional LSTM'S
        self.lstm = nn.LSTM(input_size, embed_size, n_layers,
                          dropout=dropout, bidirectional=True)
# feed forward layer;s
    def forward(self, x):       
        output, (hn, cn) = self.lstm(x)
        
# sum bidirectional outputs
        output = (output[:, :, :self.embed_size] +
                   output[:, :, self.embed_size:])
        return output, hn
#encoder output is returned and passed to the decoder


class Attn_(nn.Module):
    def __init__(self, hidden_size):
        super(Attn_, self).__init__()
        self.hidden_size = hidden_size

        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)
        

    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)  
        attn_energies = self.score(h, encoder_outputs)      
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

    
    def score(self, hidden, encoder_outputs):
   
        temp = torch.cat([hidden, encoder_outputs], dim=2)
        energy = F.relu(self.attn(temp))
        energy = energy.transpose(1, 2)  
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  
        energy = torch.bmm(v, energy)  
        return energy.squeeze(1)  


#Main Self attention class 
class Attn(nn.Module):
    def __init__(self, h_dim,c_num):
        super(Attn_, self).__init__()
        self.h_dim = h_dim
        self.v = nn.Parameter(torch.rand(h_dim))
        self.out = nn.Linear(self.h_dim, c_num)

        self.main = nn.Sequential(
            nn.Linear(h_dim, c_num),
            nn.ReLU(True),
            nn.Linear(24,1)
        )

#Actual process
    def forward(self, hidden , encoder_outputs):
        b_size = encoder_outputs.size(0)

#atten_energies are calculated using encoder outputs and hidden layers
        attn_ene = self.main(encoder_outputs.view(-1, self.h_dim)) 


#Multiplying q*k
        attn_applied = torch.bmm(attn_ene.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0)) 
        
#scaling:sqrt(size(h_dim))     
        output=attn_applied[0]/math.sqrt(self.v.size(0))
        
#softmax
        output = F.log_softmax(self.out(output[0]), dim=1).unsqueeze(2)
        return output

In [ ]:
#Decoder class 
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size,
                 dropout=0.2):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
#         self.dropout = nn.Dropout(dropout, inplace=True)
        
        ## attention layer
        self.attention = Attn_(hidden_size)
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size * 2, output_size) 
        self.sig = nn.Sigmoid()

    def forward(self, last_hidden, encoder_outputs):

# Calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
#context vector=attention weights ,ecnoder outputs

#[q*k]*v
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  
        context = context.transpose(0, 1)  
        output = self.fc(last_hidden.view(-1, 2*self.hidden_size))
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1))
#output = F.log_softmax(output, dim=1)
        return self.sig(output), attn_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):

        encoder_output, hidden = self.encoder(src) 
        output, attn_weights = self.decoder(hidden, encoder_output)

        return output

In [ ]:
#getting the encoder layer with below units
enc = Encoder(40, 256, 1).cuda()

#getting the decoder layer
dec = Decoder(256, 1).cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#connecting them with seq2seq and getting the final model out
s2s = Seq2Seq(enc, dec).to(device)
EPOCH = 15

#binary cross entropy loss since our task is classification
loss_fn = nn.BCELoss()

#learning rate 
lr = 0.001
opt_weight=-0.001
best_class_weights=[10,8,94,48]

#adam optimizer
optimizer = torch.optim.AdamW(s2s.parameters(), lr=lr)

/opt/conda/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
print(s2s)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(40, 256, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): Attn_(
      (attn): Linear(in_features=512, out_features=256, bias=True)
    )
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (out): Linear(in_features=512, out_features=1, bias=True)
    (sig): Sigmoid()
  )
)


In [ ]:
dataiter = iter(train_loader)
data = dataiter.next()
images, labels = data['data'],data['label']
print(images.shape)
print(labels.shape)

torch.Size([12, 40, 8064])
torch.Size([12, 1])


In [ ]:
#Training the model
for epoch in range(15):

    #model.train
    s2s.train()
    train_loss = 0
    
    ## training bathces in gpu
    for i, batch in enumerate(train_loader):
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()
        
        optimizer.zero_grad()
        output = s2s(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    ## evaluating the trained model on validation set
    s2s.eval()
    val_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = s2s(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, (opt_weight*train_loss)/len(train_loader), (opt_weight*val_loss)/len(val_loader))) 

Epoch : 0 train_loss : 0.30612760579295806 val_loss : 0.4290378889604049
Epoch : 1 train_loss : 0.4120200511133948 val_loss : 0.4290378889604049
Epoch : 2 train_loss : 0.4147645458842433 val_loss : 0.4290378889604049
Epoch : 3 train_loss : 0.4142776340218477 val_loss : 0.4290378889604049
Epoch : 4 train_loss : 0.41259594016851386 val_loss : 0.4290378889604049
Epoch : 5 train_loss : 0.4134486240120821 val_loss : 0.4290378889604049
Epoch : 6 train_loss : 0.4137219103436138 val_loss : 0.4290378889604049
Epoch : 7 train_loss : 0.41267346084949585 val_loss : 0.4290378889604049
Epoch : 8 train_loss : 0.4138711331389671 val_loss : 0.4290378889604049
Epoch : 9 train_loss : 0.4133401276344477 val_loss : 0.4290378889604049
Epoch : 10 train_loss : 0.4127664833955987 val_loss : 0.4290378889604049
Epoch : 11 train_loss : 0.41342927444812866 val_loss : 0.4290378889604049
Epoch : 12 train_loss : 0.41276066802268807 val_loss : 0.4290378889604049
Epoch : 13 train_loss : 0.412809117516806 val_loss : 0.4

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

#Calculating the metrics
fin_targets = []
fin_outputs = []

with torch.no_grad():
    for i, batch in enumerate(train_loader):

        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label']
        output = s2s(data)
        fin_targets.append(np.asarray(label.numpy(),dtype=np.int))
        fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))
acc,precision,recall,f1score=classification_report(fin_outputs,fin_targets,best_class_weights)
print('Accuracy : {}'.format(acc))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(f1score))

Accuracy : 0.811
Precision: 0.889
Recall: 0.847
F1score: 0.8
